In [1]:
import sys
import os
import torch
from torch import optim
from time import time


In [2]:
tile2vec_dir = '/workspace/tile2vec-noise'
sys.path.append('../')
sys.path.append(tile2vec_dir)

In [3]:
from src.datasets import TileTripletsDatasetNoise, GetBands, RandomFlipAndRotate, ClipAndScale, ToFloatTensor, triplet_noise_dataloader
from src.tilenet import make_tilenet_noise

In [4]:
from src.training import prep_triplets, train_triplet_noise_epoch

# Step 1. Download triplets from bucket

Using the download link, unzip triplets into the directory /tile2vec/data/triplets.

# Step 2. Set up dataloader

In [5]:
# Environment stuff
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda = torch.cuda.is_available()

Set up the dataloader for training.

In [6]:
# Change these arguments to match your directory and desired parameters
img_type = 'naip'
tile_dir = '/workspace/tile2vec-noise/data/triplets/'
bands = 4
augment = True
batch_size = 15
shuffle = True
num_workers = 4
n_triplets = 100000

In [7]:
dataloader, dataset = triplet_noise_dataloader(img_type, tile_dir, bands=bands, augment=augment,
                                batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, 
                                n_triplets=n_triplets, pairs_only=True)
print('Dataloader set up complete.')

Dataloader set up complete.


# Step 3. Set up TileNet

In [8]:
in_channels = bands
z_dim = 512

In [9]:
TileNetNoise = make_tilenet_noise(in_channels=in_channels, z_dim=z_dim)
TileNetNoise.train()
if cuda: TileNetNoise.cuda()
print('TileNet set up complete.')

TileNet set up complete.


Set up optimizer.

In [10]:
lr = 1e-3
optimizer = optim.Adam(TileNetNoise.parameters(), lr=lr, betas=(0.5, 0.999))

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20], gamma=0.5)

# Step 4. Train model!

In [11]:
epochs = 50
margin = 10
alpha = 1.0
print_every = 10000
save_models = True
sampling=True

Define the directory for saving models.

In [12]:
model_dir = '/workspace/tile2vec-noise/models/'
if not os.path.exists(model_dir): os.makedirs(model_dir)

In [13]:
t0 = time()
results_fn = "results.txt"
with open(results_fn, 'w') as file:
    print('Begin training.................')
    for epoch in range(0, epochs):
        (avg_loss, avg_l_a, avg_l_n, avg_l_d, avg_noise) = train_triplet_noise_epoch(
            TileNetNoise, cuda, dataloader, dataset, batch_size, optimizer, scheduler, epoch+1, ut=3, margin=margin, alpha=alpha,
            print_every=print_every, t0=t0, sampling=sampling)

Begin training.................
Epoch 1: [30000/100000 (30%)], Avg loss: 9.6878
Epoch 1: [60000/100000 (60%)], Avg loss: 7.6628
Epoch 1: [90000/100000 (90%)], Avg loss: 6.7581
Finished epoch 1: 816.063s
  Average loss: 2.6157
  Average l_a: 7.3546
  Average l_n: -20.9323
  Average l_d: -13.5776
  Average noise: 0.6703

Epoch 2: [30000/100000 (30%)], Avg loss: 6.0194
Epoch 2: [60000/100000 (60%)], Avg loss: 5.6501
Epoch 2: [90000/100000 (90%)], Avg loss: 5.3307
Finished epoch 2: 1634.512s
  Average loss: 1.8790
  Average l_a: 7.2300
  Average l_n: -21.5227
  Average l_d: -14.2927
  Average noise: 0.6230

Epoch 3: [30000/100000 (30%)], Avg loss: 5.1497
Epoch 3: [60000/100000 (60%)], Avg loss: 5.0451
Epoch 3: [90000/100000 (90%)], Avg loss: 5.0149
Finished epoch 3: 2444.421s
  Average loss: 1.6827
  Average l_a: 7.0962
  Average l_n: -21.5315
  Average l_d: -14.4353
  Average noise: 0.6064

Epoch 4: [30000/100000 (30%)], Avg loss: 4.7617
Epoch 4: [60000/100000 (60%)], Avg loss: 4.6851
Epo

Epoch 29: [30000/100000 (30%)], Avg loss: 2.3407
Epoch 29: [60000/100000 (60%)], Avg loss: 2.3860
Epoch 29: [90000/100000 (90%)], Avg loss: 2.3944
Finished epoch 29: 24697.104s
  Average loss: 0.7890
  Average l_a: 5.3095
  Average l_n: -19.6715
  Average l_d: -14.3620
  Average noise: 0.4418

Epoch 30: [30000/100000 (30%)], Avg loss: 2.3323
Epoch 30: [60000/100000 (60%)], Avg loss: 2.3219
Epoch 30: [90000/100000 (90%)], Avg loss: 2.3252
Finished epoch 30: 25515.041s
  Average loss: 0.7709
  Average l_a: 5.2523
  Average l_n: -19.5893
  Average l_d: -14.3370
  Average noise: 0.4400

Epoch 31: [30000/100000 (30%)], Avg loss: 2.2840
Epoch 31: [60000/100000 (60%)], Avg loss: 2.2557
Epoch 31: [90000/100000 (90%)], Avg loss: 2.3103
Finished epoch 31: 26328.000s
  Average loss: 0.7626
  Average l_a: 5.1809
  Average l_n: -19.5088
  Average l_d: -14.3279
  Average noise: 0.4355

Epoch 32: [30000/100000 (30%)], Avg loss: 2.3049
Epoch 32: [60000/100000 (60%)], Avg loss: 2.2523
Epoch 32: [90000/

In [14]:
# Save model after last epoch
if save_models:
    model_fn = os.path.join(model_dir, 'TileNetNoise_epoch50_s1.ckpt')
    torch.save(TileNetNoise.state_dict(), model_fn)

In [15]:
'''
from tqdm import tqdm
for ind, (samples, noise) in enumerate(dataloader):
    print(ind)
    print(samples)
    print(noise)
    break
'''

'\nfrom tqdm import tqdm\nfor ind, (samples, noise) in enumerate(dataloader):\n    print(ind)\n    print(samples)\n    print(noise)\n    break\n'